## Loading google big query client to load data 

In [2]:
import sys
import pandas as pd
import logging
from typing import List
from google.cloud import bigquery
from google.oauth2 import service_account

# TODO(developer): Set key_path to the path to the service account key file.

key_path = "/Users/drewwhite/.creds/wdrew-sa.json"
credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
)

client = bigquery.Client(credentials=credentials, project=credentials.project_id,)

### BigQuery Variable ###  
PROJECT_NAME = "team-week2"
DATASET_NAME = "los_angeles"

### Create Dataset ### 
dataset_id = f"{PROJECT_NAME}.{DATASET_NAME}"
dataset = bigquery.Dataset(dataset_id)
dataset.location = "US"
dataset = client.create_dataset(dataset, exists_ok=True)

### Load Data ### 

# Set Job Config 
job_config = bigquery.LoadJobConfig(
source_format="CSV",
autodetect=True, 
create_disposition="CREATE_IF_NEEDED",
write_disposition="WRITE_TRUNCATE",
destination_table_description="Raw source data loaded to BigQuery"
)


## Reading 2010-2019 data into dataframe to verify shape

In [40]:
df = pd.read_csv("../data/LA_crime2010-2019.csv")

# replacing spaces and dashes with underscores

df.columns = df.columns.str.replace(" ", "_").str.lower()
df.columns = df.columns.str.replace("-", "_").str.lower()
df.rename(columns={'area_':'area_id'}, inplace=True)
print(df.head(5))
print(df.shape)

       dr_no               date_rptd                date_occ  time_occ  \
0    1307355  02/20/2010 12:00:00 AM  02/20/2010 12:00:00 AM      1350   
1   11401303  09/13/2010 12:00:00 AM  09/12/2010 12:00:00 AM        45   
2   70309629  08/09/2010 12:00:00 AM  08/09/2010 12:00:00 AM      1515   
3   90631215  01/05/2010 12:00:00 AM  01/05/2010 12:00:00 AM       150   
4  100100501  01/03/2010 12:00:00 AM  01/02/2010 12:00:00 AM      2100   

   area_id  area_name  rpt_dist_no  part_1_2  crm_cd  \
0       13     Newton         1385         2     900   
1       14    Pacific         1485         2     740   
2       13     Newton         1324         2     946   
3        6  Hollywood          646         2     900   
4        1    Central          176         1     122   

                                         crm_cd_desc  ... status  \
0                           VIOLATION OF COURT ORDER  ...     AA   
1  VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...  ...     IC   
2             

## Pushing 2010-2019 to bigquery

In [42]:
import numpy as np

# large data frame use code below to write in chunks

for df_chunk in np.array_split(df, 5):
  job_config = bigquery.LoadJobConfig()
  job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
  job = client.load_table_from_dataframe(df_chunk, f"{PROJECT_NAME}.{DATASET_NAME}.LA_2010_2019", job_config=job_config)
  job.result()

## Reading 2020-Present data into dataframe to verify shape 

In [38]:
df = pd.read_csv("../data/LA_crime2020-present.csv")

# replacing spaces and dashes with underscores

df.columns = df.columns.str.replace(" ", "_").str.lower()
df.columns = df.columns.str.replace("-", "_").str.lower()
df.rename(columns={'area':'area_id'}, inplace=True)
print(df.head(5))
print(df.shape)

       dr_no               date_rptd                date_occ  time_occ  \
0   10304468  01/08/2020 12:00:00 AM  01/08/2020 12:00:00 AM      2230   
1  190101086  01/02/2020 12:00:00 AM  01/01/2020 12:00:00 AM       330   
2  201220752  09/16/2020 12:00:00 AM  09/16/2020 12:00:00 AM      1230   
3  191501505  01/01/2020 12:00:00 AM  01/01/2020 12:00:00 AM      1730   
4  191921269  01/01/2020 12:00:00 AM  01/01/2020 12:00:00 AM       415   

   area_id    area_name  rpt_dist_no  part_1_2  crm_cd  \
0        3    Southwest          377         2     624   
1        1      Central          163         2     624   
2       12  77th Street         1259         2     745   
3       15  N Hollywood         1543         2     745   
4       19      Mission         1998         2     740   

                                         crm_cd_desc  ... status  status_desc  \
0                           BATTERY - SIMPLE ASSAULT  ...     AO  Adult Other   
1                           BATTERY - SIMPLE

## Pushing 2020-Present to bigquery

In [39]:
job = client.load_table_from_dataframe(df, f"{PROJECT_NAME}.{DATASET_NAME}.LA_2020_PRESENT", job_config=job_config)
job.result()

LoadJob<project=team-week2, location=US, id=2f730620-1707-46a7-949e-34bf92e885da>

## Concat data sets for better visualization

In [3]:
df1 = pd.read_csv("../data/LA_crime2010-2019.csv")
df2 = pd.read_csv("../data/LA_crime2020-present.csv")

# replacing spaces and dashes with underscores

df1.columns = df1.columns.str.replace(" ", "_").str.lower()
df1.columns = df1.columns.str.replace("-", "_").str.lower()
df1.rename(columns={'area_':'area_id'}, inplace=True)

df2.columns = df2.columns.str.replace(" ", "_").str.lower()
df2.columns = df2.columns.str.replace("-", "_").str.lower()
df2.rename(columns={'area_':'area_id'}, inplace=True)

print(df1.shape)
print(df2.shape)

(2118203, 28)
(326213, 28)


In [4]:
frames = [df1, df2]

combined = pd.concat(frames)

print(combined.shape)

(2444416, 29)


In [5]:
import numpy as np

# large data frame use code below to write in chunks

for df_chunk in np.array_split(combined, 5):
  job_config = bigquery.LoadJobConfig()
  job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
  job = client.load_table_from_dataframe(df_chunk, f"{PROJECT_NAME}.{DATASET_NAME}.LA_2010_PRESENT", job_config=job_config)
  job.result()